Импортируйте данные data.csv.zip (ссылка) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [2]:
import pandas as pd

In [3]:
retail = pd.read_csv('https://stepik.org/media/attachments/lesson/361623/data.csv.zip', 
                     compression = 'zip', encoding = 'ISO-8859-1')
retail_columns = retail.columns

In [6]:
nrows_originals = retail.shape[0]
retail.drop_duplicates(inplace = True)
nrows_without_duplicates = retail.shape[0]
nrows_originals - nrows_without_duplicates

5268

⭐️Задание со звёздочкой!⭐️

Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 

Сколько всего заказов отменили пользователи?

In [16]:
retail.InvoiceNo.str.startswith('C').sum()

9251

Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [19]:
retail.query('Quantity > 0').shape

(526054, 8)

💀 Сложное задание! 💀

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

quantile
query

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [21]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [37]:
german_buyers = retail.query('Country == "Germany"').groupby('CustomerID', as_index = False) \
                                    .agg({'InvoiceNo': pd.Series.nunique})

quantile_80 = german_buyers.InvoiceNo.quantile(q = 0.8)

In [38]:
germany_top = german_buyers.query('InvoiceNo > @quantile_80').CustomerID
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

⭐️Задание со звёздочкой!⭐️

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [42]:
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.


In [44]:
top_retail_germany.StockCode.value_counts()

POST     219
22326     62
22423     55
22328     45
22467     37
        ... 
71279      1
22702      1
22635      1
22273      1
20963      1
Name: StockCode, Length: 1168, dtype: int64

In [45]:
top_retail_germany.groupby('StockCode').agg({'Description':'count'}).sort_values('Description')

,Description
StockCode,
22548,1
22948,1
84800L,1
22956,1
22174,1
...,...
22467,37
22328,45
22423,55


Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [47]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [50]:
revenues_invoices = retail.groupby('InvoiceNo', as_index = False) \
    .agg({'Revenue':'sum'}) \
    .sort_values('Revenue', ascending = False)

In [53]:
print(revenues_invoices.InvoiceNo[:5].str.cat(sep = ', '))

581483, 541431, 574941, 576365, 556444
